In [ ]:
import pygame, sys
import neat
from settings import * 
from level_2 import Level
from ui import UI
import math
import os
import time
import pickle

#create some global variables
player_coor = None 
goal_coor = None
distance = None
run_2 = None
run = None

player_initial_pos_x = 64


class Game:
    def __init__(self, screen): #added self.screen for level and UI 

        # game attributes
        self.max_level = 2
        self.max_health = 100
        self.cur_health = 100
        self.coins = 0
        self.fitness = 0 
        
        #screen 
        self.screen = screen

        # audio 
        self.level_bg_music = pygame.mixer.Sound('../audio/level_music.wav')
        self.level_bg_music.set_volume(0.5)
        self.overworld_bg_music = pygame.mixer.Sound('../audio/overworld_music.wav')
        self.overworld_bg_music.set_volume(0.3)
        
        #level creation
        # self.level = Level(random.randint(0,2),screen, self.change_coins, self.change_health)
        self.level = Level(0,self.screen, self.change_coins, self.change_health)
        
        # user interface 
        self.ui = UI(self.screen)

        #count 
        self.count = 0

    def change_coins(self,amount):
        self.coins += amount
        self.fitness += 10

    def change_health(self,amount):
        self.cur_health += amount
        self.fitness -= 5

    def check_game_over(self):
        global run 
        if self.cur_health <= 0 or self.count == 600:
            self.fitness -= 100
            run = False
            # pygame.quit()
            # sys.exit()


    #new code added
    def fitness_distance(self):
        global player_coor, goal_coor, player_initial_pos_x, distance #updating fitness only for the change in the x direction 
        # dist = abs(goal_coor[0] - player_coor[0])
        # self.fitness += 1/(dist+1)
        # u += 1/(dist+1)
        # print(u)
        if player_initial_pos_x < self.level.player.sprite.rect.x:
            # dist = abs(goal_coor[0] - player_coor[0])
            self.fitness += 2000/(distance+1)
            # u += 100/(dist+1)
            # print(self.fitness)
            # print(1000/(distance+1))           
        else:
            self.fitness -= 500/(distance+1)

        if abs(player_initial_pos_x - self.level.player.sprite.rect.x) <= 20:
            self.count += 1
        else:
            self.count = 0

        # print(self.fitness)

    def run(self):
        global run_2, player_coor, goal_coor, distance
        
        self.level.run()
        level_fitness, run_2, player_coor, goal_coor = self.level.update_var()
        distance = math.sqrt((player_coor[0] - goal_coor[0])**2 + (player_coor[1] - goal_coor[1])**2)
        self.fitness += level_fitness
        self.ui.show_health(self.cur_health,self.max_health)
        self.ui.show_coins(self.coins)
        self.check_game_over()
        self.fitness_distance()
    
    def update_fitness(self):
        return self.fitness
        
# Pygame setup
# pygame.init()
# screen = pygame.display.set_mode((screen_width,screen_height))
# clock = pygame.time.Clock()
# game = Game()


def main(genomes, config): #the genomes are the entities present in a generation 
    global distance, player_coor, goal_coor, run, run_2, player_initial_pos_x

    nets = [] #list of neural networks for each genome(player) for a single gen
    ge = [] #list of genomes for a single gen 
    games = [] #list of games(in which each player runs)

    genome_counter = 0 

    for _, g in genomes:
        genome_counter += 1
        pygame.init()
        screen = pygame.display.set_mode((screen_width,screen_height))
        clock = pygame.time.Clock()

        ####Create custom key events 
        # key_event_space = pygame.event.Event(pygame.KEYDOWN, key=pygame.K_SPACE)
        # key_event_left = pygame.event.Event(pygame.KEYDOWN, key=pygame.K_LEFT)
        # key_event_right = pygame.event.Event(pygame.KEYDOWN, key=pygame.K_RIGHT)
        
        player_coor = None 
        goal_coor = None
        distance = None
        run_2 = True
        run = True 
        player_intial_pos_x = 64

        penalty = None

        net = neat.nn.FeedForwardNetwork.create(g, config) #create the neural network for the genome 
        nets.append(net) #append the nn in the nets 
        game = Game(screen)
        games.append(game)
        g.fitness = 0 #initialize the fitness function 
        ge.append(g) #append the genome in the list of genomes

        while run and run_2:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    penalty_time = pygame.time.get_ticks()
                    if penalty_time < 3000:
                        penalty = 1
                    elif penalty_time < 10000 and penalty_time >= 3000:
                        penalty = 2
                    elif penalty_time < 20000 and penalty_time >= 10000:
                        penalty = 3
                    else:
                        penalty = 4
                    run = False
        
            # print(pygame.time.get_ticks()) -----------------------------------------------------------------

            screen.fill('grey')
            game.run()
            g.fitness = game.update_fitness()

            output = net.activate((player_coor[0], player_coor[1], goal_coor[0], goal_coor[1],abs(player_coor[0]-goal_coor[0]),distance,int(game.level.player.sprite.on_ground),int(game.level.player.sprite.facing_right)))
            #now get the decision , which you will get

            # print(player_coor[0], player_coor[1], goal_coor[0], goal_coor[1],abs(player_coor[0]-goal_coor[0]),distance,int(game.level.player.sprite.on_ground),int(game.level.player.sprite.facing_right))
            decision = output.index(max(output))
            # print(decision, genome_counter) --------------------------------------------------------------
            # if decision == 0:
            #     game.level.player.sprite.direction.x = 0
            #     game.level.horizontal_movement_collision()
            # elif decision == 1 and game.level.player.sprite.on_ground:
            #     # pygame.event.post(key_event_space)
            #     game.level.player.sprite.jump()
            #     game.level.player.sprite.create_jump_particles(game.level.player.sprite.rect.midbottom)
            # elif decision == 2:
            #     # pygame.event.post(key_event_left)
            #     game.level.player.sprite.direction.x = -1
            #     game.level.horizontal_movement_collision()
            #     game.level.player.sprite.facing_right = False
            # else:
            #     # pygame.event.post(key_event_right)
            #     game.level.player.sprite.direction.x = 1
            #     game.level.horizontal_movement_collision()
            #     game.level.player.sprite.facing_right = True
            if decision == 0 and game.level.player.sprite.on_ground:
                # pygame.event.post(key_event_space)
                game.level.player.sprite.jump()
                game.level.player.sprite.create_jump_particles(game.level.player.sprite.rect.midbottom)
            elif decision == 1:
                # pygame.event.post(key_event_left)
                game.level.player.sprite.direction.x = -1
                game.level.horizontal_movement_collision(4)
                game.level.scroll_x()
                game.level.player.sprite.facing_right = False
            elif decision == 2:
                # pygame.event.post(key_event_right)
                game.level.player.sprite.direction.x = 1
                game.level.horizontal_movement_collision(4)
                game.level.scroll_x()
                game.level.player.sprite.facing_right = True


            #update the player_initial_pos_per_frame
            player_initial_pos_x = game.level.player.sprite.rect.x 
                
            
            pygame.display.update()
            clock.tick(60)
        
        if penalty == 1:
            g.fitness -= 300
        elif penalty == 2:
            g.fitness -= 100
        elif penalty == 3:
            g.fitness -= 50
        elif penalty == 4:
            g.fitness -= 10
        else:
            pass
        pygame.quit()
        # print(g.fitness) #-----------------------------------------------------------------------------

        # time.sleep(5) #-----------------------------------------------------------------------------

#function to run the neat algo 
def run(config_path):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

    # p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-27')
    p = neat.Population(config)

    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(1))

    winner = p.run(main, 50) #run upto 50 generations

    with open("best.pickle", "wb") as f:
        pickle.dump(winner, f)
        
if __name__ == "__main__":
    # local_dir = os.path.dirname(__file__)


    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward.txt")
    run(config_path)     

In [ ]:
import pygame, sys
from settings import * 
from level import Level
from overworld import Overworld
from ui import UI

class Game:
	def __init__(self):

		# game attributes
		self.max_level = 2
		self.max_health = 100
		self.cur_health = 100
		self.coins = 0
		
		# audio 
		self.level_bg_music = pygame.mixer.Sound('../audio/level_music.wav')
		self.overworld_bg_music = pygame.mixer.Sound('../audio/overworld_music.wav')

		# overworld creation
		self.overworld = Overworld(0,self.max_level,screen,self.create_level)
		self.status = 'overworld'
		self.overworld_bg_music.play(loops = -1)

		# user interface 
		self.ui = UI(screen)


	def create_level(self,current_level):
		self.level = Level(current_level,screen,self.create_overworld,self.change_coins,self.change_health)
		self.status = 'level'
		self.overworld_bg_music.stop()
		self.level_bg_music.play(loops = -1)

	def create_overworld(self,current_level,new_max_level):
		if new_max_level > self.max_level:
			self.max_level = new_max_level
		self.overworld = Overworld(current_level,self.max_level,screen,self.create_level)
		self.status = 'overworld'
		self.overworld_bg_music.play(loops = -1)
		self.level_bg_music.stop()

	def change_coins(self,amount):
		self.coins += amount

	def change_health(self,amount):
		self.cur_health += amount

	def check_game_over(self):
		if self.cur_health <= 0:
			self.cur_health = 100
			self.coins = 0
			self.max_level = 0
			self.overworld = Overworld(0,self.max_level,screen,self.create_level)
			self.status = 'overworld'
			self.level_bg_music.stop()
			self.overworld_bg_music.play(loops = -1)

	def run(self):
		if self.status == 'overworld':
			self.overworld.run()
		else:
			self.level.run()
			self.ui.show_health(self.cur_health,self.max_health)
			self.ui.show_coins(self.coins)
			self.check_game_over()

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((screen_width,screen_height))
clock = pygame.time.Clock()
game = Game()

while True:
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			pygame.quit()
			sys.exit()
	
	screen.fill('grey')
	game.run()

	pygame.display.update()
	clock.tick(60)

In [ ]:
import pygame 
import sys
from support import import_csv_layout, import_cut_graphics
from settings import tile_size, screen_height, screen_width
from tiles import Tile, StaticTile, Crate, Coin, Palm
from enemy import Enemy
from decoration import Sky, Water, Clouds
from player import Player
from particles import ParticleEffect
from game_data import levels
import time

class Level:
    ####--> 404
    # def __init__(self,current_level,surface,create_overworld,change_coins,change_health):
    def __init__(self,current_level,surface,change_coins,change_health):
        # general setup
        self.display_surface = surface
        self.world_shift = 0
        self.current_x = None
        self.fitness = 0
        self.game_loop_status = True

        # audio 
        self.coin_sound = pygame.mixer.Sound('../audio/effects/coin.wav')
        self.stomp_sound = pygame.mixer.Sound('../audio/effects/stomp.wav')

        # overworld connection 
        ####-->404
        # self.create_overworld = create_overworld
        self.current_level = current_level
        level_data = levels[self.current_level]
        self.new_max_level = level_data['unlock']

        # player 
        player_layout = import_csv_layout(level_data['player'])
        self.player = pygame.sprite.GroupSingle()
        self.goal = pygame.sprite.GroupSingle()
        self.player_setup(player_layout,change_health)

        # user interface 
        self.change_coins = change_coins

        # dust 
        self.dust_sprite = pygame.sprite.GroupSingle()
        self.player_on_ground = False

        # explosion particles 
        self.explosion_sprites = pygame.sprite.Group()

        # terrain setup
        terrain_layout = import_csv_layout(level_data['terrain'])
        self.terrain_sprites = self.create_tile_group(terrain_layout,'terrain')

        # grass setup 
        grass_layout = import_csv_layout(level_data['grass'])
        self.grass_sprites = self.create_tile_group(grass_layout,'grass')

        # crates 
        crate_layout = import_csv_layout(level_data['crates'])
        self.crate_sprites = self.create_tile_group(crate_layout,'crates')

        # coins 
        coin_layout = import_csv_layout(level_data['coins'])
        self.coin_sprites = self.create_tile_group(coin_layout,'coins')

        # foreground palms 
        fg_palm_layout = import_csv_layout(level_data['fg palms'])
        self.fg_palm_sprites = self.create_tile_group(fg_palm_layout,'fg palms')

        # background palms 
        bg_palm_layout = import_csv_layout(level_data['bg palms'])
        self.bg_palm_sprites = self.create_tile_group(bg_palm_layout,'bg palms')

        # enemy 
        enemy_layout = import_csv_layout(level_data['enemies'])
        self.enemy_sprites = self.create_tile_group(enemy_layout,'enemies')

        # constraint 
        constraint_layout = import_csv_layout(level_data['constraints'])
        self.constraint_sprites = self.create_tile_group(constraint_layout,'constraint')

        # decoration 
        self.sky = Sky(8)
        level_width = len(terrain_layout[0]) * tile_size
        self.water = Water(screen_height - 20,level_width)
        self.clouds = Clouds(400,level_width,30)

    def create_tile_group(self,layout,type):
        sprite_group = pygame.sprite.Group()

        for row_index, row in enumerate(layout):
            for col_index,val in enumerate(row):
                if val != '-1':
                    x = col_index * tile_size
                    y = row_index * tile_size

                    if type == 'terrain':
                        terrain_tile_list = import_cut_graphics('../graphics/terrain/terrain_tiles.png')
                        tile_surface = terrain_tile_list[int(val)]
                        sprite = StaticTile(tile_size,x,y,tile_surface)
                        
                    if type == 'grass':
                        grass_tile_list = import_cut_graphics('../graphics/decoration/grass/grass.png')
                        tile_surface = grass_tile_list[int(val)]
                        sprite = StaticTile(tile_size,x,y,tile_surface)
                    
                    if type == 'crates':
                        sprite = Crate(tile_size,x,y)

                    if type == 'coins':
                        if val == '0': sprite = Coin(tile_size,x,y,'../graphics/coins/gold',5)
                        if val == '1': sprite = Coin(tile_size,x,y,'../graphics/coins/silver',1)

                    if type == 'fg palms':
                        if val == '0': sprite = Palm(tile_size,x,y,'../graphics/terrain/palm_small',38)
                        if val == '1': sprite = Palm(tile_size,x,y,'../graphics/terrain/palm_large',64)

                    if type == 'bg palms':
                        sprite = Palm(tile_size,x,y,'../graphics/terrain/palm_bg',64)

                    if type == 'enemies':
                        sprite = Enemy(tile_size,x,y)

                    if type == 'constraint':
                        sprite = Tile(tile_size,x,y)

                    sprite_group.add(sprite)
        
        return sprite_group

    def player_setup(self,layout,change_health):
        for row_index, row in enumerate(layout):
            for col_index,val in enumerate(row):
                x = col_index * tile_size
                y = row_index * tile_size
                if val == '0':
                    sprite = Player((x,y),self.display_surface,self.create_jump_particles,change_health)
                    self.player.add(sprite)
                if val == '1':
                    hat_surface = pygame.image.load('../graphics/character/hat.png').convert_alpha()
                    sprite = StaticTile(tile_size,x,y,hat_surface)
                    self.goal.add(sprite)

    def enemy_collision_reverse(self):
        for enemy in self.enemy_sprites.sprites():
            if pygame.sprite.spritecollide(enemy,self.constraint_sprites,False):
                enemy.reverse()

    def create_jump_particles(self,pos):
        if self.player.sprite.facing_right:
            pos -= pygame.math.Vector2(10,5)
        else:
            pos += pygame.math.Vector2(10,-5)
        jump_particle_sprite = ParticleEffect(pos,'jump')
        self.dust_sprite.add(jump_particle_sprite)

    def horizontal_movement_collision(self, given_speed = None):
        player = self.player.sprite
        ####
        if given_speed == None:
            req_speed = player.speed
        else:
            req_speed = given_speed
        player.collision_rect.x += player.direction.x * req_speed
        collidable_sprites = self.terrain_sprites.sprites() + self.crate_sprites.sprites() + self.fg_palm_sprites.sprites()
        for sprite in collidable_sprites:
            if sprite.rect.colliderect(player.collision_rect):
                # print("collide")
                #just did this now
                self.fitness -= 0.00001
                # print(self.fitness) --------------------------------------------------------------
                if player.direction.x < 0: 
                    player.collision_rect.left = sprite.rect.right
                    player.on_left = True
                    self.current_x = player.rect.left
                elif player.direction.x > 0:
                    player.collision_rect.right = sprite.rect.left
                    player.on_right = True
                    self.current_x = player.rect.right

    def vertical_movement_collision(self):
        player = self.player.sprite
        player.apply_gravity()
        collidable_sprites = self.terrain_sprites.sprites() + self.crate_sprites.sprites() + self.fg_palm_sprites.sprites()

        for sprite in collidable_sprites:
            if sprite.rect.colliderect(player.collision_rect):
                if player.direction.y > 0: 
                    player.collision_rect.bottom = sprite.rect.top
                    player.direction.y = 0
                    player.on_ground = True
                elif player.direction.y < 0:
                    player.collision_rect.top = sprite.rect.bottom
                    player.direction.y = 0
                    player.on_ceiling = True

        if player.on_ground and player.direction.y < 0 or player.direction.y > 1:
            player.on_ground = False

    def scroll_x(self):
        player = self.player.sprite
        player_x = player.rect.centerx
        direction_x = player.direction.x

        if player_x < screen_width / 4 and direction_x < 0:
            self.world_shift = 8
            player.speed = 0
        elif player_x > screen_width - (screen_width / 4) and direction_x > 0:
            self.world_shift = -8
            player.speed = 0
        else:
            self.world_shift = 0
            player.speed = 8

    def get_player_on_ground(self):
        if self.player.sprite.on_ground:
            self.player_on_ground = True
        else:
            self.player_on_ground = False

    def create_landing_dust(self):
        if not self.player_on_ground and self.player.sprite.on_ground and not self.dust_sprite.sprites():
            if self.player.sprite.facing_right:
                offset = pygame.math.Vector2(10,15)
            else:
                offset = pygame.math.Vector2(-10,15)
            fall_dust_particle = ParticleEffect(self.player.sprite.rect.midbottom - offset,'land')
            self.dust_sprite.add(fall_dust_particle)

    def check_death(self):
        if self.player.sprite.rect.top > screen_height or self.player.sprite.rect.bottom < 0:
            self.fitness -= 200 
            # self.create_overworld(self.current_level,0) --> 404
            self.game_loop_status = False 

            #debug-code-404
            # pygame.quit()
            # sys.exit()
            ###
            
    def check_win(self):
        if pygame.sprite.spritecollide(self.player.sprite,self.goal,False):
            self.fitness += 500
            # self.create_overworld(self.current_level,self.new_max_level) --> 404
            
            self.game_loop_status = False

            #debug-code-404
            # pygame.quit()
            # sys.exit()	
            ###	
            
    def check_coin_collisions(self):
        collided_coins = pygame.sprite.spritecollide(self.player.sprite,self.coin_sprites,True)
        if collided_coins:
            self.coin_sound.play()
            for coin in collided_coins:
                self.change_coins(coin.value)

    def check_enemy_collisions(self):
        enemy_collisions = pygame.sprite.spritecollide(self.player.sprite,self.enemy_sprites,False)

        if enemy_collisions:
            for enemy in enemy_collisions:
                enemy_center = enemy.rect.centery
                enemy_top = enemy.rect.top
                player_bottom = self.player.sprite.rect.bottom
                if enemy_top < player_bottom < enemy_center and self.player.sprite.direction.y >= 0:
                    self.stomp_sound.play()
                    self.player.sprite.direction.y = -15
                    explosion_sprite = ParticleEffect(enemy.rect.center,'explosion')
                    self.explosion_sprites.add(explosion_sprite)
                    enemy.kill()
                else:
                    self.player.sprite.get_damage()
                    
    def update_var(self):
        return self.fitness, self.game_loop_status, self.player.sprite.rect.center, self.goal.sprite.rect.center 
    
    # def update_game_condition(self):
    #     return self.game_loop_status

    def run(self):
        # run the entire game / level 
        
        # sky 
        self.sky.draw(self.display_surface)
        self.clouds.draw(self.display_surface,self.world_shift)
        
        # background palms
        self.bg_palm_sprites.update(self.world_shift)
        self.bg_palm_sprites.draw(self.display_surface) 

        # dust particles 
        self.dust_sprite.update(self.world_shift)
        self.dust_sprite.draw(self.display_surface)
        
        # terrain 
        self.terrain_sprites.update(self.world_shift)
        self.terrain_sprites.draw(self.display_surface)
        
        # enemy 
        self.enemy_sprites.update(self.world_shift)
        self.constraint_sprites.update(self.world_shift)
        self.enemy_collision_reverse()
        self.enemy_sprites.draw(self.display_surface)
        self.explosion_sprites.update(self.world_shift)
        self.explosion_sprites.draw(self.display_surface)

        # crate 
        self.crate_sprites.update(self.world_shift)
        self.crate_sprites.draw(self.display_surface)

        # grass
        self.grass_sprites.update(self.world_shift)
        self.grass_sprites.draw(self.display_surface)

        # coins 
        self.coin_sprites.update(self.world_shift)
        self.coin_sprites.draw(self.display_surface)

        # foreground palms
        self.fg_palm_sprites.update(self.world_shift)
        self.fg_palm_sprites.draw(self.display_surface)

        # player sprites
        self.player.update()
        self.horizontal_movement_collision()
        
        self.get_player_on_ground()
        self.vertical_movement_collision()
        self.create_landing_dust()
        
        self.scroll_x()
        self.player.draw(self.display_surface)
        self.goal.update(self.world_shift)
        self.goal.draw(self.display_surface)

        self.check_death()
        self.check_win()

        self.check_coin_collisions()
        self.check_enemy_collisions()

        # water 
        self.water.draw(self.display_surface,self.world_shift)
